# Answer to...
![Title](image-20220823-135919.png)
##  by Knight Lab Fellows (Joon Park and Cathy He) and Joe Germuska

### Click here to check out the [MYSTERY]("https://mystery.knightlab.com/")!
#### or follow along!
(Note: no outputs are displayed. Look at the outputs yourself and inquire! I have put my thoughts down after each query!)

Enjoy!

### The Synopsis:
![synopsis](image-20220823-141656.png)

### This query was provided:
(used to see the names of the tables)
![Table name query](image-20220823-142301.png)

### They also provided a query to see what data types are in each table.
You can change the table name in the quotes to see the information on other tables.
![data types query](image-20220823-142621.png)

### Here is the Relational Database Model:
![RDB Model](image-20220823-143442.png)

### It said to start off with the crime_scene_report with specific filters: 
### Jan. 15, 2018, murder crime, located in SQL City

In [ ]:


step_1 = _deepnote_execute_sql("""select *
from crime_scene_report
where date = 20180115 and type = 'murder' and city = 'SQL City';
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_1

3 murders
(and the last murder has a clue):
- 1st witness lives on last house Northwestern Dr (no name)
- 2nd witness from Franklin Ave (Annabel)

### Lets see who lives on these streets:

In [ ]:


step_2 = _deepnote_execute_sql("""select name, address_number, address_street_name
from person
-- where address_street_name = 'Franklin Ave and name like 'Annabel_Miller'
-- ^ to confirm Annabel's information
where address_street_name = 'Northwestern Dr'
order by address_number desc;
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_2

- Annabel Miller 103 Franklin Ave 
- A ton of names on Northwestern Dr
- but possible witnesses from 'last house' of Northwestern Dr
- Morty Schapiro or Kinsey Erickson (the potential residents of 'last house' on street)

### Lets look through interviews to find a clue that the witnesses might have said:

In [ ]:


step_3 = _deepnote_execute_sql("""select gfnm.name, i.transcript
from get_fit_now_member gfnm left join interview i on gfnm.person_id = i.person_id
where gfnm.name like 'Annabel%';
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_3

Annabel saw the murder happen and recognized the killer from the gym on Jan. 9th = 20180109

### Need to see if id is the same as membership_id:

In [ ]:


step_4 = _deepnote_execute_sql("""select id, name
from get_fit_now_member
where name like '%Annabel%';
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_4

id is the same as membership_id so we can just join the tables to get names

### Lets see who went to the gym on January 9th, 2018:

In [ ]:


step_5 = _deepnote_execute_sql("""select membership_id, gfnm.name, check_in_time, check_out_time
from get_fit_now_check_in gfnci 
	left join get_fit_now_member gfnm 
	on gfnci.membership_id = gfnm.id 
where check_in_date = 20180109;
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_5

### Lets do the same thing but join the street address and narrow down the time to evenings when Annabel is at the gym:
(Note: I did this thinking that the addresses had something to do with it)
 

In [ ]:


step_6 = _deepnote_execute_sql("""select membership_id, gfnm.name, check_in_time, check_out_time, p.address_street_name
from get_fit_now_check_in gfnci 
	left join get_fit_now_member gfnm 
	on gfnci.membership_id = gfnm.id 
	left join person p 
	on gfnm.person_id = p.id
where check_in_date = 20180109 and check_in_time >= 1530 and check_out_time <= 1730;
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_6

2 others were at the gym: Joe Germuska (lol a contributor) and Jeremy Bowers

### I want to find out if they were interviewed:

In [ ]:


step_7 = _deepnote_execute_sql("""select p.name, i.transcript
from person p left join interview i on p.id = i.person_id
where (p.name like 'Annabel_Miller' or p.name like 'Joe_Germuska' or p.name like 'Jeremy_Bowers');
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_7

Clue from Jeremy: 
- hired by rich woman: height 65"-67", has red hair, owns tesla model s, attended sql symphony concert 3 times in December 2017

### 2 ways we can go about this:
First way 
- we look through the FB events and see who went to the concert 3 times and investigate their wealth
Second way
- we investigate their wealth and then investigate the wealthy suspect


In [ ]:


step_8 = _deepnote_execute_sql("""select p.name, count(p.name) quantity, fec.event_name
from facebook_event_checkin fec left join person p on fec.person_id = p.id
where fec.event_name = 'SQL Symphony Concert' and fec.date like '201712%'
group by p.name
order by quantity desc;
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_8

Only 2 people who went 3 times:
- Bryan Pardo and Miranda Priestly
- Jeremy said that a rich woman hired him. 

### Well, I don't want to assume gender, so let's find their SSN and get their income:

In [ ]:


step_9 = _deepnote_execute_sql("""select i.annual_income, p.name
from income i left join person p on i.ssn = p.ssn
where (p.name like 'Miranda_Priestly' or p.name like 'Bryan%');
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_9

Miranda is def rich w/ 310k income, but Bryan does not have a SSN...
Interesting...

### Need to confirm that Bryan has a SSN and is in Person table:

In [ ]:


step_10 = _deepnote_execute_sql("""select name, ssn
from person
where name like 'Bryan_Pardo';
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_10

#### Bryan's SSN is 816663882, so let's see if an income matches the SSN:

In [ ]:


step_11 = _deepnote_execute_sql("""select i.annual_income, p.name
from income i left join person p on i.ssn = p.ssn
where i.ssn = 816663882;
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
step_11

No result. So, we move on to Miranda.

### Lets match Miranda's description with what Jeremy said by joining the person table and driver's license:

In [ ]:


df_13 = _deepnote_execute_sql("""select p.name, d.height, d.hair_color, d.gender, d.car_make, d.car_model
from person p left join drivers_license d on p.license_id = d.id
where name like 'Miranda_Priestly';
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_13

Everything matches Jeremy's description!

### With that being said: 

Jeremy is probably the killer since they were recognized by Annabel at the gym. 
Jeremy also said he was hired by a rich woman. 
Rich woman concluded to be Miranda and confirmed description.
Miranda is the real mastermind.

#### Entering Jeremy's name into solution:
![Soltution_jeremy](image-20220826-125043.png)


#### Entering Miranda's name into solution:
![Solution_miranda](image-20220826-125251.png)

![Credits](image-20220823-140351.png)

![Credits](image-20220823-140351.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=da733383-ae0d-4402-8489-9080cb4a0d93' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>